### Reconsidering Databases:

During the development, we used SQLite database and that served us well.<br>

Although that SQLite is a good DBMS it is not good for heavy loads, hence you may want to choose another DBMS for deployment.<br>

There are a lot of options when we are talking about DataBase Management systems, and you can use almost all of them with django.<br>


To choose a DBMS, the first question you need to ask yourself is, should I use a SQL database or a NoSQL database?<br>

I don't want to talk about their deferences, but I want you to know that Django-models only support SQL databases and not NoSQL databases.

### Django is NOT a Webserver:

- Django is a framework for python
    - It knows how to work with incoming request and create responses
- Django is not the webserver, the software that listens for the requests or handle any other server tasks is NOT django.
    - You would need a webserver for that for e.q., 'apachi'

The asgi.py and wsgi.py files are for this purpose.<br>
    - Theya re the entry point of our website, where the webserver should point at.

Why do we have 2 files and not 1?<br>
Well, Django currently supports two interfaces, WSGI and ASGI.
When you want to deploy your project you should check with your host to see which interface they support! it is usualy WSGI, but you should check to make sure!<br>
I personaly havent seen a host with ASGI interface.<br>
If you go through the Django documentation you will know the steps you need to take for each interface.<br>

### python manage.py runserver:

NOTE: Django does not serve static or uploaded files automatically. Yes in the development server it does. but not an actual server.<br>

We can configure django to server such files via urls.py<br>
    - This is O.K. for small website, but you should be aware that it is not performance-optimized.<br>
The second option is to not let django take care of those files, and instead to configure a web server and the django app separately<br>

The third option is to use a dedicated service/ server for static and uploaded files.
    - In this cenario we would have 2 servers, one for handling Django, and 1 for hadnleing static files (This is very optimized! but costly and complex).

### Hosting:

You can use any hosting... I haven't decided what to use! I will update This section later.


### Static Fiels:
<br>
Static files are the fiels that you as the developer ship with you python-django application on the server.<br>
These static fiels could be images, documents, css and html files.<br>
Django DOES NOT automatically serves static files.<br>

In you settings.py:<br>
    - There is a STATICFIELS_DIRS list, that is important for django so that it is aware of where your static files are. By default it will look in your app folders and look for static folder dir, but it will also will look every directory you mention here.<br>
    - It is important for django to know where they are for two reasons:<br>
        - In development it needs to know where theya re because it serves them.<br>
        - In production it needs to collect them for you. When it gets to production, you want to collect all your static fiels from all the locations you put them and move them to one folder. This folder then gets served by either django (if you config it) or some dedicated host.<br>
        - The collection part is something that we havent done yet, because durnig development we didnt need to, because django did it for usautomatically.<br>
        - For production we need to do it manualy:<br>
            - You could go ahead and copy all you static files into one folder, but this would be cumbersome.<br>
            - Another option that you have is a setting that you could add to your django settings.py file.<br>
                - Add STATIC_ROOT option to your settings.py and pass a folder name of your choice.<br>
                - This should be an absolute address, so use BASE_DIR / 'folder-name'<br>
                - IT looks like MEDIA_ROOT setting, doesnt it? Yes it does. Let me tell you this, they are doing the same thing.<br>
                - You should keep the STATIC_URL settings! Don't delete it.<br>
                - You don't Want to have the same folder for your STATIC fiels and MEDIA fiels<br>
                    - Technically you could, but doing it will cause security problems. Let me explain why: Your static files could contain javascript fiels for example, as long as they are your code as the developer there is no problem, hoever if a user fins a way of uploading their own javascript code with some cruppted code, they your website will start sending them to all the users and thats a bad thing!!!! <br>
            - After Adding The STATIC_ROOT you will need to open you command prompt:<br>
                - python manage.py collectstatic<br>
                    - This command will go ahead and collect all the static fiels and put them to the folder you mentioned.<br>
                    - Never edit these files! If you want to do anything, do it on your own files and then run the command to collect them.<br>
Your STATIC_ROOT should look like the following:<br>

STATIC_ROOT = BASAE_DIR / 'staticfiels

Now, lets talk about the 3 options that we had:<br>

1. Making Django take care of static fiels:
Navigate to the root urls.py file, and add a new entry by calling static again and point at STATIC_URL and the STATIC_URL for the folder.

Your urlspy file should look like the following block:

In [ ]:
from django.contrib import admin
from django.urls import path, include
from django.conf.urls.static import static
from django.conf import settings

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", include("blog.urls"))
] + static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
+ static(settings.STATIC_URL, document_root=settings.STATIC_ROOT)

You should make migrations and migrate now! Also make sure that you have a superuser!<br>

Now we are ready to prepare this application for deployment.

1. The first step is to make sure that the dependency packages are avalible on the server!
    - We used Django package and also pillow package!
    - To insure that our host has the packages is something that is dependent on the host youa re using.
    - Most python hosting providers support a special text file, which is when added to your project simply list the packages you need, the hosting server then automatcally makes sure that those packages are availble for you.
    - You can create this file yourself! or ask python to create this file for you.
        -What you need to do is to use python -m pip freeze > requirements.txt
        This will create the file for you, make sure you run it on the venv! otherwise all the packages that you have on your system will be listed!
2. Navigate to settings.py
    - Locate ALLOWED_HOSTS option
        - You need to add all the hosts that can send requests to this application
        - This is not the address of the users!
        - Here we need to enter the doman address of the server
            - Often you know this in advance
            - You can do one thing if you dont know this! you can use environment variables here so that the value changes later when you deploy this application.
            - From os import getenv
            - Now you can use it to to populate ALLOWED_HOST
            - getenv('APP_HOST') [You can put any name!]
    - Also you need to keep remove the security key from your settings.py file and store it somewhere else!
        - If you keep it there, you should make sure that you dont expose it on github.
        - I myself use getenv('SECRET_KEY') and then add the key to my system with the key value.
        - DONT DELETE THE SECRETKEY! KEEP IT SOMEWHERE ELSE! DONT LOSE IT.
3. You also need to pass FALSE to DEBUG (it should be True for development only)
    Here is what I changed in settings.py file:

In [ ]:
from os import getenv

SECRET_KEY = getenv("SECRET_KEY")

DEBUG = False

ALLOWED_HOSTS = [
    getenv('APP_HOST')
]